## Understanding Clouds from Satellite Images

This project will utilize a CNN to attempt to classify images of clouds from satellite data.

### Base model

In [2]:
# Import necessary libraries

from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import os
import shutil
import pandas as pd
import random
from tqdm import tqdm_notebook
import cv2
import gc

import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import backend as K
from keras import models
from keras import layers
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

import warnings
warnings.filterwarnings("ignore")

AttributeError: module 'keras_applications' has no attribute 'set_keras_submodules'

### Pre-processing

In [4]:
tr = pd.read_csv('data/train.csv')
print(len(tr))
tr.head()

22184


,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [5]:
def rle2mask(rle, imgshape):
    width = imgshape[0]
    height= imgshape[1]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return np.flipud( np.rot90( mask.reshape(height, width), k=1 ) )

In [6]:
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [7]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [8]:
img_size = 256

In [9]:
img_names_all = tr['Image_Label'].apply(lambda x: x.split('_')[0]).unique()
len(img_names_all)

5546

### Generator

In [10]:
new_ep = False
def keras_generator(batch_size):  
    global new_ep
    while True:   
        
        x_batch = []
        y_batch = []        
        for _ in range(batch_size):                         
            if new_ep == True:
                img_names =  img_names_all
                new_ep = False
            
            fn = img_names[random.randrange(0, len(img_names))]                                       

            img = cv2.imread(path + 'train_images/'+ fn)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)                       
            masks = []
            for rle in tr[tr['Image_Label'].apply(lambda x: x.split('_')[0]) == fn]['EncodedPixels']:                
                if pd.isnull(rle):
                    mask = np.zeros((img_size, img_size))
                else:
                    mask = rle2mask(rle, img.shape)
                    mask = cv2.resize(mask, (img_size, img_size))
                masks.append(mask)                                        
            img = cv2.resize(img, (img_size, img_size))            
            x_batch += [img]
            y_batch += [masks] 

            img_names = img_names[img_names != fn]   
        
        x_batch = np.array(x_batch)
        y_batch = np.transpose(np.array(y_batch), (0, 2, 3, 1))        

        yield x_batch, y_batch

### Create Model

In [11]:
import segmentation_models as sm

AttributeError: module 'keras_applications' has no attribute 'set_keras_submodules'

In [17]:
BACKBONE = 'resnet50'
preprocess_input = sm.backbones.get_preprocessing(BACKBONE)

model = sm.Unet(
           encoder_name=BACKBONE, 
           classes=4,
           activation='sigmoid',
           input_shape=(img_size, img_size, 3))

model.compile(optimizer=optimizers.Adam(lr=9e-3), loss=bce_dice_loss)

NameError: name 'sm' is not defined